In [ ]:
import os
import logging
logging.getLogger('tensorflow').disabled = True

try:
    os.chdir('cifar-vs-tensorflow2')
except:
    pass

import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
if not tf.test.is_gpu_available():
    print("GPU is not available, change Runtime!")

import matplotlib.pyplot as plt
from tqdm import tqdm, tqdm_notebook
from tensorflow.keras.utils import plot_model
from solver import Solver
from cnn_model import *
import resent2 


In [ ]:
ds = tfds.load('cifar10', as_supervised=True)
std = tf.reshape((0.2023, 0.1994, 0.2010), shape=(1, 1, 3))
mean= tf.reshape((0.4914, 0.4822, 0.4465), shape=(1, 1, 3))

def train_prep(x, y):
    x = tf.cast(x, tf.float32)/255.
    x = tf.image.random_flip_left_right(x)
    x = tf.image.pad_to_bounding_box(x, 4, 4, 40, 40)
    x = tf.image.random_crop(x, (32, 32, 3))
    x = (x - mean) / std
    return x, y

def valid_prep(x, y):
    x = tf.cast(x, tf.float32)/255.
    x = (x - mean) / std
    return x, y

ds['train'] = ds['train'].map(train_prep).shuffle(5000).repeat().batch(128).prefetch(-1)
ds['test'] = ds['test'].map(valid_prep).batch(512).prefetch(-1)

In [ ]:
model = resent2.cifar_resnet20('preactivated', shortcut_type='B')
training_steps = 64000
validation_interval = 2000

model.fit_generator(ds['train'], steps_per_epoch=validation_interval, epochs=training_steps//validation_interval, validation_data=ds['test'], workers=4)